In [19]:
import pandas as pd
from fuzzywuzzy import fuzz
from nameparser import HumanName

In [12]:
ppd = pd.read_csv('../../../Data/PPD/ppd_data_20200404.csv')
spec = pd.read_csv('../../../Data/PPD/speciality_id.csv')
df = pd.read_csv('../../../Data/Medscape/Memorium_USA.csv')

In [14]:
ppd

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [18]:
PPD_SPEC = pd.merge(ppd, spec, left_on='PRIM_SPEC_CD', right_on='SPEC_CD')

In [20]:
def split_names(roster_df):
    '''Splits name column into components'''
    roster_df = roster_df.drop_duplicates()
    dict_list = []
    for row in roster_df.itertuples():
        name_parsed = HumanName(row.NAME)
        name_dict = {
            'NAME':row.NAME,
            'FIRST_NAME': name_parsed.first.upper(),
            'LAST_NAME': name_parsed.last.upper(),
            'MIDDLE_NAME': name_parsed.middle.upper(),
            'SUFFIX':name_parsed.suffix.upper(),
            'NICKNAME':name_parsed.nickname.upper(),
            'TITLE':name_parsed.title.upper(),
        }
        dict_list.append(name_dict)
    name_df = pd.DataFrame(dict_list)
    new_df = pd.merge(name_df, roster_df, on='NAME')
    return new_df

In [22]:
df = split_names(df)

In [30]:
bad_spec_words = [
    'Nurse',
    'Vet',
    'Transport',
    'Assistant',
    'Receptionist'
]

In [65]:
mes =[]
for row in df.itertuples():
    NAME = row.NAME
    ME = 'None'
    keep = True
    for word in bad_spec_words:
        if word in row.SPECIALTY:
            keep = False
    if keep == True:
        new_df = PPD_SPEC[(PPD_SPEC.FIRST_NAME==row.FIRST_NAME)&(PPD_SPEC.LAST_NAME==row.LAST_NAME)]
        if len(new_df)==0:
            pass
        elif len(new_df)>1:
            new_df = new_df[new_df.BIRTH_YEAR.isin([2019.0-int(row.AGE), 2020.0-int(row.AGE)])]
            if len(new_df)>1:
                print('wtf')
        if len(new_df)==1:
            ME = list(new_df.ME)[0]
    mes.append(ME)

In [66]:
df['ME']=mes

In [69]:
df[df.ME!='None']

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,AGE,SPECIALTY,CITY,STATE,COUNTRY,LOCATION,ME
6,Marques Bolden,MARQUES,BOLDEN,,,,,35,Hospitalist,South Bend,Indiana,USA,None,1720102877
15,James T. Goodrich,JAMES,GOODRICH,T.,,,,73,Neurosurgeon Director of the Division of Pedia...,New York City,New York,USA,Pediatrics,3501801606
17,Alex Hsu,ALEX,HSU,,,,,67,Internal Medicine,Margate,Florida,USA,Northwest Medical Center,1401820236
25,John F. Murray,JOHN,MURRAY,F.,,,,92,Pulmonary and Critical Care Physician,San Francisco,California,USA,San Francisco General Hospital,511530385
27,Tomas Pattugalan,TOMAS,PATTUGALAN,,,,,70,Internal Medicine Physician,Queens,New York,USA,None,74810740365
28,Mark Respler,MARK,RESPLER,,,,,66,Urologist,Brooklyn,New York,USA,None,3508801512
30,Stephen Schwartz,STEPHEN,SCHWARTZ,,,,,78,Pathologist,Seattle,Washington,USA,None,2405670502


In [70]:
PPD_SPEC.to_csv('../../../Data/PPD/ppd_spec.csv',index=False)

In [71]:
PPD_SPEC[PPD_SPEC.ME==511530385]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,SPEC_CD,SPEC_ID,DESC
1241460,511530385,A,1,1.0,JOHN F MURRAY MD,MURRAY,JOHN,F,NaN,NaN,...,C014,MURRAY,FREDERIC,JOHN,NaN,NaN,NaN,PUD,1847,PULMONARY DISEASE
